# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai
!pip install chromadb
!pip install ag2[retrievechat]

In [1]:
import json
import os

with open('../config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
import autogen
autogen.__version__

'0.9.1post0'

# RAG System

In [20]:
import os 
os.getcwd()

'/home/bni/postdoc_MIT/LLMHackathon/codes'

In [3]:
from autogen import ConversableAgent, LLMConfig
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

In [24]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import GroupChat, GroupChatManager

from chromadb.utils import embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                 api_key=config["api_key"],
                model_name="text-embedding-3-small"
            )

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="assistant. You are a helpful assistant. You retrieve knowledge from a text. You should pay attention to all the details, specially quantitative data.",
    llm_config=llm_config,
)

reviewer = RetrieveAssistantAgent(
    name="reviewer",
    system_message='''reviewer. double-check the response from the assistant for correctness.
Return 'TERMINATE' in the end when the task is over.''',
    llm_config=llm_config,
)

ragproxyagent = RetrieveUserProxyAgent(
    human_input_mode="NEVER",
    name="ragproxyagent",
    retrieve_config={
        "task": "qa",
        "docs_path": "./code_dir/SciAgents.pdf",
        "embedding_function": openai_ef,
        "model": "gpt-4o",
        "overwrite": True,
        "get_or_create": True,
    },
    code_execution_config=False,
)

groupchat_rag = GroupChat(
    agents=[assistant, reviewer, ragproxyagent, #sequence_retriever,
               ], messages=[], max_round=20,
    speaker_selection_method='auto',
)

manager_rag = GroupChatManager(groupchat=groupchat_rag, llm_config=llm_config,
    system_message='You dynamically select a speaker.')

/tmp/ipykernel_240129/2822795229.py:12: DeprecationWarning: The RetrieveAssistantAgent is deprecated. Please use the AssistantAgent instead.
  assistant = RetrieveAssistantAgent(
/tmp/ipykernel_240129/2822795229.py:18: DeprecationWarning: The RetrieveAssistantAgent is deprecated. Please use the AssistantAgent instead.
  reviewer = RetrieveAssistantAgent(


In [25]:
def retrieve_knowledge(msg: str):
    res = ragproxyagent.initiate_chat(
        manager_rag,
        message=ragproxyagent.message_generator,
        problem=msg,
        clear_history=True,
        silent=False,
        summary_method="reflection_with_llm",
        summary_args={"summary_prompt":"Fully return the retrieved knowledge"}
    )

    return res.summary

In [26]:
retrieve_knowledge('tell me about SciAgents')

Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	SCIAGENTS : AUTOMATING SCIENTIFIC DISCOVERY THROUGH ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2025-05-28 07:52:24,253 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 46 chunks.
2025-05-28 07:52:24,259 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.


VectorDB returns doc_ids:  [['50e43885', 'e1d48bc3', '6e88957b', 'a31fe033', 'b619051d', 'b240b988', '3487bac0', '48abf238', '5844bb73', '8d12879a', '29cfd757', 'c3f8f316', '98ce573e', '6d944194', 'cddde65f', '66d3fea2', 'c9c1c5e7', 'e52c70a3', 'eb54a217', '1988495f']]
Adding content of doc 50e43885 to context.
Adding content of doc e1d48bc3 to context.
ragproxyagent (to chat_manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: tell me about SciAgents

Context is: achieve the desired optical
properties
Energy Efficiency Energy-intensive, high-
temperature processing
(boiling in Na2CO3
solution at ˜100°C).
Low-temperature process-
ing below 50 °C, reduc-
ing energy consumption by
˜30%.
The energy savings can be
quantified 

'SciAgents is an automated multi-agent AI framework designed to enhance scientific discovery through the integration of large language models (LLMs) and a comprehensive knowledge graph. It effectively generates and refines research hypotheses by navigating complex relationships between scientific concepts. The system utilizes a structured approach to construct knowledge graphs, comprising nodes and relationships, which serve as input for generative models. These models expand on the graph, providing detailed definitions and explanations of concepts and generating structured scientific hypotheses, which include fields like mechanisms, unexpected properties, and comparisons.\n\nBy leveraging generative models, SciAgents enables a dynamic exploration of scientific inquiry, predicting outcomes, exploring mechanisms, and proposing innovative ideas based on graph relationships. The framework further employs a process of prompting and systematic expansion of research aspects, compilation into